# **Load & Quick Scan**

### *Task 1: Load the dataset into a DataFrame*

In [1]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
data_path = Path('consumer_complaints_unclean.csv')
df = pd.read_csv(data_path)
df_raw = df.copy()

### *Task 2: Check shape and column data types*

In [2]:
print(df.shape)
print(df.dtypes)

(2040, 16)
Complaint ID                      int64
Date Received                    object
Date Sent to Company             object
Product                          object
Sub-product                      object
Issue                            object
Company                          object
State                            object
ZIP code                         object
City                             object
Company response to consumer     object
Timely response?                 object
Consumer disputed?               object
Latitude                        float64
Longitude                       float64
Status                           object
dtype: object


### *Task 3: Display the first five rows*

In [3]:
df.head()

Complaint ID Date Received Date Sent to Company        Product  \
0        100000    2023-11-10           2024-01-01       mortgage   
1        100001    2023-03-12           2023-03-21       MORTGAGE   
2        100002    2023-10-20           2023-11-27                  
3        100003    2023-02-17           2023-03-13  Student  loan   
4        100004    2023-11-21           2023-11-23    credit card   

       Sub-product                  Issue        Company State ZIP code  \
0    Interest rate                           Metro Loans    NY    75285   
1      Collections  Collection harassment            NaN    FL    43383   
2    Interest rate                  Other            NaN    NY    43864   
3              NaN                   Fees    Metro Loans    GA      123   
4  Adjustable loan  Collection harassment  United Credit    PA    62086   

         City Company response to consumer Timely response?  \
0   new york                           NaN                N   
1       miami                          NaN              NaN   
2     Unknown                      Unknown            FALSE   
3   new york                           NaN          Unknown   
4    New York                      Unknown              Yes   

  Consumer disputed?   Latitude   Longitude    Status  
0                NaN  42.312932  -91.462816    Closed  
1                  N  42.236514 -103.703588  Resolved  
2                NaN  41.279018  -94.258898    Closed  
3                 No  34.924120  -85.770266            
4                NaN  35.643178  -86.962992   Unknown

# **Missing Values**

### *Task 4: Compute percentage of missing values per column*

In [4]:
missing_pct = df.isna().mean().mul(100).sort_values(ascending=False)
missing_pct

Company response to consumer    33.529412
Sub-product                     26.568627
Consumer disputed?              24.019608
Company                         17.598039
City                            14.313725
State                           13.970588
Timely response?                12.156863
Product                          3.872549
Issue                            3.431373
Date Received                    0.000000
Date Sent to Company             0.000000
Complaint ID                     0.000000
ZIP code                         0.000000
Latitude                         0.000000
Longitude                        0.000000
Status                           0.000000
dtype: float64

### *Task 5: Identify columns with missingness above eighty percent*

In [5]:
high_missing = missing_pct[missing_pct > 80]
high_missing

Series([], dtype: float64)

### *Task 6: Decide handling strategy for high-missing columns*

In [6]:
columns_to_drop = high_missing.index.tolist()
df = df.drop(columns=columns_to_drop)
df.shape

(2040, 16)

# **Duplicates**

### *Task 7: Count duplicate rows*

In [7]:
duplicate_rows = df.duplicated().sum()
duplicate_rows

np.int64(40)

### *Task 8: Drop duplicates by complaint identifier keeping the latest date received*

In [8]:
shape_before_dedup = df.shape
df = df.sort_values(by='Date Received', key=lambda s: pd.to_datetime(s, errors='coerce'))
df = df.drop_duplicates(subset='Complaint ID', keep='last')
shape_after_dedup = df.shape

### *Task 9: Report dataset shape before and after dropping duplicates*

In [9]:
print(shape_before_dedup)
print(shape_after_dedup)

(2040, 16)
(1885, 16)


# **Data Types & Parsing**

### *Task 10: Convert date fields to datetime*

In [10]:
df['Date Received'] = pd.to_datetime(df['Date Received'], errors='coerce')
df['Date Sent to Company'] = pd.to_datetime(df['Date Sent to Company'], errors='coerce')
df[['Date Received', 'Date Sent to Company']].dtypes

Date Received           datetime64[ns]
Date Sent to Company    datetime64[ns]
dtype: object

### *Task 11: Convert geographic and postal fields to numeric*

In [11]:
df['ZIP code'] = pd.to_numeric(df['ZIP code'], errors='coerce')
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
df[['ZIP code', 'Latitude', 'Longitude']].dtypes

ZIP code     float64
Latitude     float64
Longitude    float64
dtype: object

### *Task 12: Convert categorical location fields to string dtype*

In [12]:
df['Product'] = df['Product'].astype('string')
df['State'] = df['State'].astype('string')
df['City'] = df['City'].astype('string')
df[['Product', 'State', 'City']].dtypes

Product    string[python]
State      string[python]
City       string[python]
dtype: object

# **Renaming & Category Standardization**

### *Task 13: Rename columns to lowercase snake case*

In [13]:
df = df.rename(columns=lambda c: re.sub(r'[^0-9a-zA-Z]+', '_', c.strip().lower()).strip('_'))
df.columns.tolist()

['complaint_id',
 'date_received',
 'date_sent_to_company',
 'product',
 'sub_product',
 'issue',
 'company',
 'state',
 'zip_code',
 'city',
 'company_response_to_consumer',
 'timely_response',
 'consumer_disputed',
 'latitude',
 'longitude',
 'status']

### *Task 14: Trim spaces and title case selected categorical columns*

In [14]:
df['product'] = df['product'].str.strip().str.title()
df['state'] = df['state'].str.strip().str.title()
df['city'] = df['city'].str.strip().str.title()
df[['product', 'state', 'city']].head()

product state       city
1709  Credit Reporting    Il    Chicago
1248      Student Loan    Ny    Unknown
1169      Bank Account    Il   New York
1242       Credit Card    Nc  Charlotte
1499          Mortgage    Mi   New York

### *Task 15: Harmonize inconsistent product spellings*

In [15]:
product_original = df['product'].copy()
product_map = {
    'credit card': 'Credit Card',
    'credit card or prepaid card': 'Credit Card Or Prepaid Card',
    'bank account or service': 'Bank Account Or Service',
    'consumer loan': 'Consumer Loan',
    'debt collection': 'Debt Collection',
    'student loan': 'Student Loan',
    'mortgage': 'Mortgage',
    'vehicle loan or lease': 'Vehicle Loan Or Lease',
    'payday loan': 'Payday Loan',
    'credit reporting': 'Credit Reporting',
    'money transfers': 'Money Transfers'
}
df['product'] = df['product'].str.lower().map(product_map).fillna(product_original)
df['product'].unique()

array(['Credit Reporting', 'Student Loan', 'Bank Account', 'Credit Card',
       'Mortgage', 'Credit  Card', 'Unknown', 'Debt Collection',
       'Student  Loan', 'Debt  Collection', <NA>, ''], dtype=object)

# **Null-like Tokens, Outliers & Sanity Checks**

### *Task 16: Replace null-like tokens with missing values*

In [16]:
null_tokens = ['N/A', 'n/a', 'Unknown', 'unknown', 'UNKNOWN', '']
df = df.replace(null_tokens, pd.NA)
df.isna().sum()

complaint_id                      0
date_received                     0
date_sent_to_company              0
product                         144
sub_product                     494
issue                            95
company                         605
state                           535
zip_code                        159
city                            462
company_response_to_consumer    927
timely_response                 473
consumer_disputed               455
latitude                          0
longitude                         0
status                          313
dtype: int64

### *Task 17: Detect latitude and longitude outliers using IQR*

In [17]:
lat_q1 = df['latitude'].quantile(0.25)
lat_q3 = df['latitude'].quantile(0.75)
lat_iqr = lat_q3 - lat_q1
lon_q1 = df['longitude'].quantile(0.25)
lon_q3 = df['longitude'].quantile(0.75)
lon_iqr = lon_q3 - lon_q1
lat_bounds = (lat_q1 - 1.5 * lat_iqr, lat_q3 + 1.5 * lat_iqr)
lon_bounds = (lon_q1 - 1.5 * lon_iqr, lon_q3 + 1.5 * lon_iqr)
lat_long_outliers = df[(df['latitude'] < lat_bounds[0]) | (df['latitude'] > lat_bounds[1]) | (df['longitude'] < lon_bounds[0]) | (df['longitude'] > lon_bounds[1])]
lat_long_outliers[['latitude', 'longitude']]

latitude   longitude
423   200.000000  -98.387197
1027  999.000000  -83.962667
618  -200.000000  -82.441493
975    32.525855  400.000000
1196 -200.000000 -114.081161
...          ...         ...
208    34.150904  400.000000
1570  999.000000  -91.745148
1460   59.575738  -88.135644
369    48.794043 -400.000000
107    18.303842 -100.709548

[67 rows x 2 columns]

### *Task 18: Validate five-digit ZIP codes*

In [18]:
zip_int = pd.to_numeric(df['zip_code'], errors='coerce').astype('Int64')
zip_str = zip_int.astype('string').str.zfill(5)
zip_mask = zip_str.str.len().eq(5) & zip_str.str.isnumeric()
df['zip_code'] = zip_str.where(zip_mask, pd.NA)
df['zip_code'].head()

1709    00123
1248    75201
1169    40722
1242    00000
1499    54760
Name: zip_code, dtype: string

# **Derived Feature & Export**

### *Task 19: Compute response time in days*

In [19]:
df['response_time_days'] = (df['date_sent_to_company'] - df['date_received']).dt.days
df['response_time_days'].head()

1709    58
1248    57
1169    37
1242    10
1499    38
Name: response_time_days, dtype: int64

### *Task 20: Select useful subset of columns*

In [20]:
selected_columns = ['complaint_id', 'product', 'sub_product', 'issue', 'company', 'state', 'city', 'zip_code', 'status', 'company_response_to_consumer', 'timely_response', 'consumer_disputed', 'response_time_days']
df_subset = df[selected_columns].copy()
df_subset.head()

complaint_id           product    sub_product           issue  \
1709        101709  Credit Reporting    Collections           Other   
1248        101248      Student Loan     Fixed loan  Identity theft   
1169        101169      Bank Account     Fixed loan  Identity theft   
1242        101242       Credit Card  Interest rate  Identity theft   
1499        101499          Mortgage    Collections           Other   

                company state       city zip_code       status  \
1709  Northstar Finance    Il    Chicago    00123  In Progress   
1248         FinServ Co    Ny       <NA>    75201         Open   
1169                NaN    Il   New York    40722                
1242               <NA>    Nc  Charlotte    00000         <NA>   
1499  Northstar Finance    Mi   New York    54760         <NA>   

     company_response_to_consumer timely_response consumer_disputed  \
1709                          NaN            <NA>                 Y   
1248                         <NA>             Yes               Yes   
1169                  In progress            <NA>               NaN   
1242                       Closed             NaN                No   
1499                         <NA>               N                 Y   

      response_time_days  
1709                  58  
1248                  57  
1169                  37  
1242                  10  
1499                  38

### *Task 21: Export cleaned subset to CSV*

In [21]:
output_path = Path('consumer_complaints_cleaned.csv')
df_subset.to_csv(output_path, index=False)
output_path.exists()

True